In [1]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

%matplotlib

Using matplotlib backend: TkAgg


In [2]:
import scipy.fftpack
from scipy.signal import gaussian

In [3]:
def f_beta(a,q,beta_guess):
    return q**2/( (beta_guess+2)**2 - a - q**2/( (beta_guess+4)**2 - a - q**2/( (beta_guess+6)**2 - a - q**2/( (beta_guess+8)**2 - a - q**2/( (beta_guess+10)**2 - a - q**2/( (beta_guess+10)**2 - a -q**2/( (beta_guess+12)**2 - a ) ) ) ) ) ) )

def beta_continue_alamano(a,q,beta_guess): #a l'air ok
    return a + f_beta(a,q,beta_guess) + f_beta(a,q,-beta_guess)

# GiantMol

In [5]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide


r0 = 2.865e-3/1.14511 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+
Omega = 2.0e6*2*pi # 2.047e6
bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*100000)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.328007


In [21]:
# Give q
# compute URF, beta, UDC for spherical cloud
# compute secular frequencies

# Udc = 10.93 # 2.8628*4   9.014464
q = np.linspace(0.61,0.63,11) # np.linspace(0.54,0.70,9) # np.linspace(0.54,0.70,9)
print('q',q)
Urf = q*(m_Ca*r0**2*Omega**2)/( 4*C_e)
print('Urf =',Urf)
beta_guess = 0.423
a = 0
for k in range(750):
    beta_guess = beta_continue_alamano(a,q,beta_guess)
    beta_guess = sqrt(beta_guess)
#     print(beta_guess)
print('non adiab',beta_guess)
print('adiab    ',sqrt(a+q**2/2))
Udc_for_alpha1 = 1/6*(beta_guess*Omega/(2*pi*100000))**2
beta_guess = sqrt(a+q**2/2)
# Udc_for_alpha1 = m_Ca*d0**2/(3*kappa_simion*C_e)*(0.5*beta_guess*Omega)**2
print('Udc_for_alpha1 =',Udc_for_alpha1)
Udc = Udc_for_alpha1
# omega_z_2 = 2*C_e*(mkappa)*Udc/m_Ca/d0**2
# omega_x_2 = 2*(C_e*Urf/m_Ca/Omega/r0**2)**2
omega_z_2 = ( 2*pi*100e3 )**2 * Udc
# omega_z_2 = 2*kappa_simion*C_e/(m_Ca*d0**2) * Udc
omega_x_2 = ( beta_guess*Omega/2 )**2
omega_r_2 = omega_x_2 - 0.5*omega_z_2

qx = 4*C_e*Urf/(m_Ca*r0**2*Omega**2)
ax = 8*C_e*0/(m_Ca*r0**2*Omega**2)
az = 2*omega_z_2/Omega**2

print('alpha =',omega_z_2/omega_r_2)
print('qx =',qx)
print('ax =',ax)
print('az =',az)
print('f_z =',sqrt(omega_z_2)/2/pi)
print('f_x =',sqrt(omega_x_2)/2/pi)
print('f_r =',sqrt(omega_r_2)/2/pi)

q [0.61  0.612 0.614 0.616 0.618 0.62  0.622 0.624 0.626 0.628 0.63 ]
Urf = [62.62349193 62.82881486 63.03413778 63.23946071 63.44478363 63.65010655
 63.85542948 64.0607524  64.26607533 64.47139825 64.67672118]
non adiab [0.47166063 0.47355768 0.47545979 0.47736704 0.47927947 0.48119715
 0.48312014 0.48504851 0.48698231 0.48892162 0.49086649]
adiab     [0.43133514 0.43274935 0.43416356 0.43557778 0.43699199 0.4384062
 0.43982042 0.44123463 0.44264885 0.44406306 0.44547727]
Udc_for_alpha1 = [14.83091669 14.95045819 15.07080093 15.19195251 15.31392065 15.43671321
 15.5603382  15.68480377 15.81011822 15.93628998 16.06332767]
alpha = [1.32542665 1.32869684 1.33200183 1.33534217 1.3387184  1.34213109
 1.34558082 1.34906818 1.35259378 1.35615823 1.35976217]
qx = [0.61  0.612 0.614 0.616 0.618 0.62  0.622 0.624 0.626 0.628 0.63 ]
ax = 0.0
az = [0.07415458 0.07475229 0.075354   0.07595976 0.0765696  0.07718357
 0.07780169 0.07842402 0.07905059 0.07968145 0.08031664]
f_z = [385109.29210859 3866

In [129]:
Urf = 6.56*1e1
Udc = 16.7
# Urf = 10.266
# Udc = 0.3346
q = Urf/( (m_Ca*r0**2*Omega**2)/( 4*C_e) )
print('q =',q)
print('Urf =',Urf)
beta_guess = 0.9
a = 0
for k in range(750):
    beta_guess = beta_continue_alamano(a,q,beta_guess)
    beta_guess = sqrt(beta_guess)
#     print(beta_guess)
print('non adiab',beta_guess)
print('adiab    ',sqrt(a+q**2/2))

omega_z_2 = 2*kappa_simion*C_e/(m_Ca*d0**2) * Udc
omega_x_2 = ( beta_guess*Omega/2 )**2
omega_r_2 = omega_x_2 - 0.5*omega_z_2

qx = 4*C_e*Urf/(m_Ca*r0**2*Omega**2)
ax = 8*C_e*0/(m_Ca*r0**2*Omega**2)
az = 2*omega_z_2/Omega**2

print('alpha =',omega_z_2/omega_r_2)
print('qx =',qx)
print('ax =',ax)
print('az =',az)
print('f_z =',sqrt(omega_z_2)/2/pi)
print('f_x =',sqrt(omega_x_2)/2/pi)
print('f_r =',sqrt(omega_r_2)/2/pi)

q = 0.6389934314665643
Urf = 65.6
non adiab 0.49968269914977426
adiab     0.45183658852366904
alpha = 1.0049174774479315
qx = 0.6389934314665642
ax = 0.0
az = 0.08349999999999999
f_z = 408656.334834051
f_x = 499682.6991497742
f_r = 407655.24629226077


In [9]:
# voltages for spherical cloud
q   = 0.195001943188
Urf = m_Ca*r0**2*Omega**2/(4*C_e)*q
Udc = d0**2/r0**4*(C_e/(kappa_simion*m_Ca*Omega**2))*Urf**2
print(Urf,Udc)

20.897685901143642 2.410659528049719


In [56]:
q = linspace(0,1.5,500)

a0 = -0.5*q**2 + 7*q**4/128 - 29*q**6/2304 + 68687*q**8/18874368
b1 = 1 - q - q**2/8 + q**3/64 - q**4/1536 - 11*q**5/36864 + 49*q**6/589824 - 55*q**7/9437184 - 83*q**8/3539440

nutest = linspace(0.1,0.9,9)
print(nutest)
a = []
for nu in nutest:
    a.append( nu**2 + q**2/(2*(nu**2-1)) + (5*nu**2+7)*q**4/(32*(nu**2-1)**3*(nu**2-4)) + (9*nu**4+58*nu**2+29)*q**6/(64*(nu**2-1)**5*(nu**2-4)*(nu**2-9)) )

[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]


In [57]:
# plot with complicated scan plot only crystal part at the end

fig = figure('stab_diag',clear='True')
ax = fig.add_subplot(1, 1, 1)
fig.set_size_inches(11.69,8.27)


ax.plot(q,a0,'-',color='k',label=r'$\Phi_z^{\mathrm{quad}}$ Quadratic pot.',lw=2)
ax.plot(q,b1,'-',color='k',label='$\Phi_z^{\mathrm{GM}}$ GiantMol axial pot.',lw=2)
# ax.plot(q,-a0,'-',color='k',label=r'$\Phi_z^{\mathrm{quad}}$ Quadratic pot.',lw=2)
# ax.plot(q,-b1,'-',color='k',label='$\Phi_z^{\mathrm{GM}}$ GiantMol axial pot.',lw=2)

for k in range(len(nutest)):
    ax.plot(q,a[k],color='k',ls=':')

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.2))
# text(-0.83, 8000, f'FWHM = ${2*sqrt(2*log(2)) * sig[to_plot] * 1000:.2f}$ MHz\n        T = {temperature:.2f} K',
# rotation=0,
# horizontalalignment='left',
# verticalalignment='bottom',
# color='xkcd:vermillion',
# fontsize=17,
#      bbox=dict(facecolor='white'))

ax.set_xlim(-0.02,1.51)
ax.set_ylim(-1.05,1.05)
# ax.set_xlabel(r'$z$ [mm]')
# ax.set_ylabel('$U_{pot}$ [eV]')
# ax.legend(fontsize=16)

ax.grid()
tight_layout()

# savefig('pot_compare_x1',dpi=300)

In [39]:
# spherical cloud
alpha = 108/55

alpha_fact = sqrt(1-alpha**(-2))
fact_alpha = sqrt(alpha**(-2)-1)
if alpha<1:
    omegaz2_sur_omegau2 = -2*( math.asinh(fact_alpha) - alpha*fact_alpha ) / ( math.asinh(fact_alpha) - alpha**(-1)*fact_alpha)
else:
    omegaz2_sur_omegau2 = -2*( math.asin(alpha_fact) - alpha*alpha_fact ) / ( math.asin(alpha_fact) - alpha**(-1)*alpha_fact)
print(omegaz2_sur_omegau2)

2.1847603706437155


/tmp/ipykernel_237210/1034454679.py:5: RuntimeWarning: invalid value encountered in sqrt
  fact_alpha = sqrt(alpha**(-2)-1)


# TADOTI

In [ ]:
r0=3.93e-3 # 2.5e-3
d0=21e-2/2

Omega=5.215e6*2*pi # 2.047e6

mkappa = 9.6*1e-4

In [ ]:
Udc = 1200
Urf = 830/4
omega_z_2 = 2*C_e*mkappa*Udc/m_Ca/d0**2
omega_x_2 = 2*(C_e*Urf/m_Ca/Omega/r0**2)**2
omega_r_2 = omega_x_2 - 0.5*omega_z_2

qx = 4*C_e*Urf/(m_Ca*r0**2*Omega**2)
az = 2*omega_z_2/Omega**2

print('alpha =',omega_z_2/omega_x_2)
print('qx =',qx)
print('ax =',ax)
print('az =',az)
print('f_z =',sqrt(omega_z_2)/2/pi)
print('f_x =',sqrt(omega_x_2)/2/pi)
print('f_r =',sqrt(omega_r_2)/2/pi)